In [ ]:

# Delta merge optimization excercise
# https://stackoverflow.com/questions/77051683/performance-issue-while-merging-data-to-a-delta-lake

from delta.tables import *

sdf = spark.range(1000).withColumn("id", (rand() * 30000000 * 2).cast(IntegerType()))\
.withColumn("par", col("id")%5)\
.withColumn("ts", current_timestamp())
display(sdf)

conditions_list=["existing.id = updates.id"]
delta_table_path="/random/source/"

if (DeltaTable.isDeltaTable(spark, delta_table_path)):
    delta_table = DeltaTable.forPath(spark, delta_table_path)
    delta_table.alias("existing").merge(
    source = sdf.alias("updates"),
    condition = " AND ".join(conditions_list)
    ).whenMatchedUpdateAll(
    ).whenNotMatchedInsertAll(
    ).execute()
else:
    df.write\
        .format("delta")\
        .mode("overwrite")\
        .partitionBy("par")\
        .save("/random/source/")

# Now add partition condition like below inside your conditions_list to improve performance.
update_cols = [i[0] for i in sdf.select("par").distinct().sort("par").collect()]
par_val = ','.join(map(str,update_cols))
conditions_list=["existing.id = updates.id"]
conditions_list.append(f"existing.par in ({par_val})")
conditions_list

#Before running if i check the record give no results.
sdf.head(5)

tmp= DeltaTable.forPath(spark, delta_table_path).toDF()
display(tmp.filter(col("par")==4).filter(col("id")==11942589)) #this number is read from sdf.head(5)

#After running the merge if i check the record again it gives results.
tmp= DeltaTable.forPath(spark, delta_table_path).toDF()
display(tmp.filter(col("par")==4).filter(col("id")==11942589)) #this number is read from sdf.head(5)